In [1]:
import icechunk
import numpy as np
import virtualizarr as vz
import xarray as xr
from obstore.store import MemoryStore, S3Store
from virtualizarr.manifests.store import ObjectStoreRegistry

from hrrrparser import HRRRParser
from hrrrparser.codecs import LEVEL_COORDINATES

#### AWS PDS HRRR bucket information

In [2]:
scheme = "s3://"
bucket = "noaa-hrrr-bdp-pds"
prefix = "hrrr.20250710/conus"

#### Configure the Virtualizarr parser to pre-generate all 18 values of the step dimension for appending.

In [3]:
parser = HRRRParser(steps=18)

#### Generate the ObjectStoreRegistry to be used for virtual chunk access.

In [4]:
object_store = S3Store(
    bucket=bucket,
    skip_signature=True,
)
registry = ObjectStoreRegistry({f"{scheme}{bucket}": object_store})

In [5]:
file_urls = [
    f"{scheme}{bucket}/{prefix.rstrip('/')}/hrrr.t22z.wrfsfcf16.grib2",
    f"{scheme}{bucket}/{prefix.rstrip('/')}/hrrr.t23z.wrfsfcf16.grib2",
]
file_urls

['s3://noaa-hrrr-bdp-pds/hrrr.20250710/conus/hrrr.t22z.wrfsfcf16.grib2',
 's3://noaa-hrrr-bdp-pds/hrrr.20250710/conus/hrrr.t23z.wrfsfcf16.grib2']

#### Use intermediate memory cache for scanning
The Gribberish library will need to scan every message in the GRIB file which can generate a lot of http requests to S3 and increase latency.
We can use obstore to create a MemoryStore, read the file into that MemoryStore and scan the data in the memory store as if it were actually coming from the S3 bucket but without the latency associated with a large number of requests

In [6]:
def cache_and_open_virtual_dataset(url, scheme, bucket, loadable_variables):
    store, path_in_store = registry.resolve(url)
    memory_store = MemoryStore()
    buffer = store.get(path_in_store).bytes()
    memory_store.put(path_in_store, buffer)
    cached_reg = ObjectStoreRegistry({f"{scheme}{bucket}": memory_store})
    vds = vz.open_virtual_dataset(
        url=url,
        parser=parser,
        registry=cached_reg,
        loadable_variables=loadable_variables,
    )
    return vds

#### Eagerly load GRIB file level variables
Several of the variables are extracted from message level metadata and are the same for all messages within the GRIB file.  For efficiency we eagerly load these variables so that we can later serialize them as native chunks rather than virtual chunks.

The GribberishCodec actually reads metadata from the first message in the file to extract values for LEVEL_COORDINATES and the other coordinate variables.  This is accomplished by having special logic for these variable keys in the codec's `_decode_single` method.  Icechunk expects to use the codec specified in the variable's `serializer` encoding key when serializing data, but since this codec is only intended for reading, the `_encode_single` method is not implemented in the codec so we need to remove this `serializer` key so Icechunk just writes the value directly.

In [7]:
loadable = LEVEL_COORDINATES + ["time", "step", "latitude", "longitude"]

vds1 = cache_and_open_virtual_dataset(
    url=file_urls[1],
    scheme=scheme,
    bucket=bucket,
    loadable_variables=loadable,
)
for var in loadable:
    if var in vds1:
        del vds1[var].encoding["serializer"]

In [8]:
vds2 = cache_and_open_virtual_dataset(
    url=file_urls[0],
    scheme=scheme,
    bucket=bucket,
    loadable_variables=loadable,
)
for var in loadable:
    if var in vds2:
        del vds2[var].encoding["serializer"]

#### Configure Icechunk store
For testing we're using an in-memory Icechunk store.  We also need to configure the store so that it is authorized to read chunks data from the GRIB files stored in S3.

This is a multi-step process, we configure a `VirtualChunkContainer` that maps a url prefix to an Icechunk I/O store that can be used for reading chunks.  We also need to configure the container credentials that the store will use to authorize S3 requests.  In this case our S3 bucket allows anonymous access.

In [9]:
storage = icechunk.in_memory_storage()
config = icechunk.RepositoryConfig.default()
s3_chunk_store = icechunk.s3_store(
    region="us-east-1",
)
config.set_virtual_chunk_container(
    icechunk.VirtualChunkContainer(f"{scheme}{bucket}/", s3_chunk_store)
)
credentials = icechunk.containers_credentials(
    {f"{scheme}{bucket}/": icechunk.s3_anonymous_credentials()}
)
repo = icechunk.Repository.open_or_create(
    storage=storage, config=config, authorize_virtual_chunk_access=credentials
)
session = repo.writable_session("main")

#### Modify the time variable encoding
Icechunk doesn't yet support the extension `dtypes` recently added to Zarr so we can't directly serialize a `datetime64[s]` value.  We update the time variable encoding to use standard CF "seconds since" encoding.

In [10]:
encoding = vds1.time.encoding
encoding["units"] = "seconds since 1970-01-01"
encoding["calendar"] = "standard"
encoding["dtype"] = "int64"
vds1.time.encoding = encoding
vds2.time.encoding = encoding

#### Container validation.
Some of our `ChunkEntry`s use the Virtualizarr convention of `path=""` to represent an empty chunk.  Currently there is a small issue in `to_icechunk` which fails when validating these `ChunkEntry`s so we need to use `validate_containers=False` until a fix is applied.

In [11]:
vds1.vz.to_icechunk(session.store, validate_containers=False)

In [12]:
vds2.vz.to_icechunk(session.store, append_dim="time", validate_containers=False)

In [13]:
ds = xr.open_zarr(session.store, group="/", zarr_format=3, consolidated=False)
ds

<xarray.Dataset> Size: 113GB
Dimensions:                            (time: 2, step: 18, isobar: 7, y: 1059,
                                        x: 1799, hag: 8, entire_atm: 1, clb: 1,
                                        adiabatic_condensation_lifted: 1,
                                        hcl: 1, sfc: 1, lcl: 1, sigma: 1,
                                        cld_ceiling: 1, bndry_cloud: 1,
                                        zero_deg_isotherm: 1, htfl: 1, atm: 1,
                                        eqm: 1, clt: 1, isotherm: 2, lfc: 1,
                                        mcl: 1)
Coordinates: (12/23)
  * clb                                (clb) float64 8B 0.0
  * adiabatic_condensation_lifted      (adiabatic_condensation_lifted) float64 8B ...
    longitude                          (y, x) float64 15MB ...
  * hcl                                (hcl) float64 8B 0.0
  * sigma                              (sigma) float64 8B 0.1
  * time                               (time) datetime64[ns] 16B 2025-07-10T1...
    ...                                 ...
  * entire_atm                         (entire_atm) float64 8B 0.0
  * isotherm                           (isotherm) float64 16B 253.0 263.0
  * sfc                                (sfc) float64 8B 0.0
  * clt                                (clt) float64 8B 0.0
  * isobar                             (isobar) float64 56B 2.5e+04 ... 1e+05
  * atm                                (atm) float64 8B 0.0
Dimensions without coordinates: y, x
Data variables: (12/69)
    tmp_isobar                         (time, step, isobar, y, x) float64 4GB ...
    tmp_hag                            (time, step, hag, y, x) float64 4GB ...
    tcolg_entire_atm                   (time, step, entire_atm, y, x) float64 549MB ...
    vgrd_hag                           (time, step, hag, y, x) float64 4GB ...
    hail_sfc                           (time, step, sfc, y, x) float64 549MB ...
    lcdc_lcl                           (time, step, lcl, y, x) float64 549MB ...
    ...                                 ...
    pres_sfc                           (time, step, sfc, y, x) float64 549MB ...
    cicep_sfc                          (time, step, sfc, y, x) float64 549MB ...
    hail_atm                           (time, step, atm, y, x) float64 549MB ...
    prate_sfc                          (time, step, sfc, y, x) float64 549MB ...
    refd_hag                           (time, step, hag, y, x) float64 4GB ...
    tcdc_atm                           (time, step, atm, y, x) float64 549MB ...

In [14]:
ds["tmp_isobar"].isel(y=100, x=100, step=16).values

array([[  0.        ,   0.        , 266.54208374, 283.99029541,
        295.74993896, 290.52233887, 291.21044922],
       [  0.        ,   0.        , 266.31001282, 283.92590332,
        295.52978516, 292.79974365, 290.33868408]])

In [17]:
ds["tmp_isobar"].isel(y=100, x=100).sel(
    time=np.datetime64("2025-07-10T18:00:00.000000000"), step=np.timedelta64(16, "h")
).values

array([  0.        ,   0.        , 266.31001282, 283.92590332,
       295.52978516, 292.79974365, 290.33868408])